In [1]:
import pyspark
from pyspark.sql import SparkSession

NESSIE_URI = "http://nessie:19120/api/v1"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"

conf = (
    pyspark.SparkConf()
        .setAppName("iceberg-spark")

        # Spark + Iceberg JAR already in image, no need for jars.packages
        .set("spark.jars", "/opt/spark/jars/iceberg-spark-runtime-3.5_2.12-1.9.0-SNAPSHOT.jar")

        # Iceberg extensions
        .set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")

        # REST catalog setup
        .set("spark.sql.catalog.rest", "org.apache.iceberg.spark.SparkCatalog")
        .set("spark.sql.catalog.rest.type", "rest")
        .set("spark.sql.catalog.rest.uri", "http://iceberg-rest:8181")
        .set("spark.sql.catalog.rest.warehouse", "s3://warehouse")
        .set("spark.sql.catalog.rest.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .set("spark.sql.catalog.rest.s3.endpoint", "http://minio:9000")
        .set("spark.sql.catalog.rest.s3.path-style-access", "true")
        .set("spark.sql.catalog.rest.s3.access-key-id", MINIO_ACCESS_KEY)
        .set("spark.sql.catalog.rest.s3.secret-access-key", MINIO_SECRET_KEY)

        # NESSIE catalog config
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
        .set('spark.sql.catalog.nessie.s3.endpoint', 'http://minio:9000')
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
        #MINIO CREDENTIALS
        .set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
)

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark.sparkContext.setLogLevel("INFO")

print("✅ Spark session started")

25/04/30 19:01:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/30 19:01:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


✅ Spark session started


In [3]:
spark.sql("""create namespace nessie.default""")

DataFrame[]

In [16]:
spark.sql("drop table if exists nessie.default.flow_records")

25/04/30 17:18:51 INFO BaseMetastoreTableOperations: Refreshing table metadata from new version: s3a://warehouse/default/flow_records_66f9e8c2-3cf8-4174-9008-087167f569b4/metadata/00002-d70d30b4-d873-49ba-8c71-fedd59a210ba.metadata.json


Py4JJavaError: An error occurred while calling o78.sql.
: org.apache.iceberg.exceptions.NotFoundException: Location does not exist: s3a://warehouse/default/flow_records_66f9e8c2-3cf8-4174-9008-087167f569b4/metadata/00002-d70d30b4-d873-49ba-8c71-fedd59a210ba.metadata.json
	at org.apache.iceberg.aws.s3.S3InputStream.openStream(S3InputStream.java:242)
	at org.apache.iceberg.aws.s3.S3InputStream.openStream(S3InputStream.java:225)
	at org.apache.iceberg.aws.s3.S3InputStream.positionStream(S3InputStream.java:221)
	at org.apache.iceberg.aws.s3.S3InputStream.read(S3InputStream.java:143)
	at org.apache.iceberg.shaded.com.fasterxml.jackson.core.json.ByteSourceJsonBootstrapper.ensureLoaded(ByteSourceJsonBootstrapper.java:539)
	at org.apache.iceberg.shaded.com.fasterxml.jackson.core.json.ByteSourceJsonBootstrapper.detectEncoding(ByteSourceJsonBootstrapper.java:133)
	at org.apache.iceberg.shaded.com.fasterxml.jackson.core.json.ByteSourceJsonBootstrapper.constructParser(ByteSourceJsonBootstrapper.ja

In [18]:
spark.sql("""
CREATE TABLE nessie.default.flow_records_without_sort (
  flow_id STRING,
  network_id STRING,
  created_at TIMESTAMP,
  bytes_sent BIGINT
)
USING iceberg
PARTITIONED BY (
  identity(network_id),
  days(created_at),
  bucket(4, flow_id)
)
""")

25/04/30 17:20:02 INFO BaseMetastoreCatalog: Table properties set at catalog level through catalog properties: {gc.enabled=false, write.metadata.delete-after-commit.enabled=false}
25/04/30 17:20:02 INFO BaseMetastoreCatalog: Table properties enforced at catalog level through catalog properties: {}
25/04/30 17:20:02 INFO NessieIcebergClient: Committed 'default.flow_records_without_sort' against 'Branch{name=main, metadata=null, hash=45af5afc08a900a3b70144c532200f0fecea9c8fb12d87bcb4df7b036ac6ff75}', expected commit-id was '2ce1318280bbcabb94321f62da2fbf565fdf549bcbbb017da491528bdca208ef'
25/04/30 17:20:02 INFO BaseMetastoreTableOperations: Successfully committed to table default.flow_records_without_sort in 28 ms
25/04/30 17:20:02 INFO BaseMetastoreTableOperations: Refreshing table metadata from new version: s3a://warehouse/default/flow_records_without_sort_16539f19-2094-41da-b2c1-54ed2c99af17/metadata/00000-a54182f0-abda-416c-8429-a58edac8547d.metadata.json
25/04/30 17:20:02 INFO Nessi

DataFrame[]

In [7]:
spark.sql("describe table extended nessie.default.flow_records").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|             flow_id|              string|   NULL|
|          network_id|              string|   NULL|
|          created_at|           timestamp|   NULL|
|          bytes_sent|              bigint|   NULL|
|                    |                    |       |
|      # Partitioning|                    |       |
|              Part 0|          network_id|       |
|              Part 1|    days(created_at)|       |
|              Part 2|  bucket(4, flow_id)|       |
|                    |                    |       |
|  # Metadata Columns|                    |       |
|            _spec_id|                 int|       |
|          _partition|struct<network_id...|       |
|               _file|              string|       |
|                _pos|              bigint|       |
|            _deleted|             boolean|       |
|           

In [14]:
spark.sql("""
ALTER TABLE nessie.default.flow_records
WRITE ORDERED BY (network_id, created_at)
""")

25/04/30 17:01:41 INFO BaseMetastoreTableOperations: Refreshing table metadata from new version: s3a://warehouse/default/flow_records_66f9e8c2-3cf8-4174-9008-087167f569b4/metadata/00000-8f6125f7-4695-4c33-b6b0-236c7a4be06b.metadata.json
25/04/30 17:01:41 INFO NessieUtil: loadTableMetadata for 'default.flow_records' from location 's3a://warehouse/default/flow_records_66f9e8c2-3cf8-4174-9008-087167f569b4/metadata/00000-8f6125f7-4695-4c33-b6b0-236c7a4be06b.metadata.json' at 'Branch{name=main, metadata=null, hash=f2be538541aae1bcd477db47247aa285b111ff78a927c0fce3d0938b96d34149}'
25/04/30 17:01:41 INFO BaseMetastoreCatalog: Table loaded by catalog: nessie.default.flow_records
25/04/30 17:01:41 INFO NessieIcebergClient: Committed 'default.flow_records' against 'Branch{name=main, metadata=null, hash=fb78c42c701d55befb0451add6329028b68f7679b46be21e7b99577bb952dfd0}', expected commit-id was 'f2be538541aae1bcd477db47247aa285b111ff78a927c0fce3d0938b96d34149'
25/04/30 17:01:41 INFO BaseMetastoreTa

DataFrame[]

In [ ]:
spark.sql("""
ALTER TABLE nessie.default.flow_records
WRITE ORDERED BY (network_id, created_at)
""")

In [19]:
from pyspark.sql.functions import col, expr
from datetime import datetime, timedelta
import random

sample_data = [
    (
        f"flow_{i}",
        random.choice(["net1", "net2"]),
        datetime(2025, 4, 30, random.randint(0, 23), random.randint(0, 59)),
        random.randint(1000, 100000)
    )
    for i in range(10000)
]

df = spark.createDataFrame(sample_data, ["flow_id", "network_id", "created_at", "bytes_sent"])
df.writeTo("nessie.default.flow_records_without_sort").append()

25/04/30 17:20:19 INFO DAGScheduler: Registering RDD 18 (append at NativeMethodAccessorImpl.java:0) as input to shuffle 1
25/04/30 17:20:19 INFO DAGScheduler: Got map stage job 3 (append at NativeMethodAccessorImpl.java:0) with 12 output partitions
25/04/30 17:20:19 INFO DAGScheduler: Final stage: ShuffleMapStage 4 (append at NativeMethodAccessorImpl.java:0)
25/04/30 17:20:19 INFO DAGScheduler: Parents of final stage: List()
25/04/30 17:20:19 INFO DAGScheduler: Missing parents: List()
25/04/30 17:20:19 INFO DAGScheduler: Submitting ShuffleMapStage 4 (MapPartitionsRDD[18] at append at NativeMethodAccessorImpl.java:0), which has no missing parents
25/04/30 17:20:19 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 17.3 KiB, free 434.4 MiB)
25/04/30 17:20:19 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 8.9 KiB, free 434.4 MiB)
25/04/30 17:20:19 INFO BlockManagerInfo: Added broadcast_4_piece0 in memory on fa5472e59cd7:450

# demo write distributed by partition
### this is beneficial if the partitions are small and almost equally distributed so that each spark task gets a partition otherwise a task may get overloaded with one partition data and the spark default write implementation is hash, so it would parallelize better

In [2]:
spark.sql("""create namespace rest.default""")

25/04/30 19:01:33 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
25/04/30 19:01:33 INFO SharedState: Warehouse path is 'file:/opt/spark/home/iceberg/notebooks/spark-warehouse'.
25/04/30 19:01:33 INFO AuthManagers: Loading AuthManager implementation: org.apache.iceberg.rest.auth.NoopAuthManager
25/04/30 19:01:33 INFO CatalogUtil: Loading custom FileIO implementation: org.apache.iceberg.aws.s3.S3FileIO


AnalysisException: [SCHEMA_ALREADY_EXISTS] Cannot create schema `default` because it already exists.
Choose a different name, drop the existing schema, or add the IF NOT EXISTS clause to tolerate pre-existing schema.

In [3]:
spark.sql("""
CREATE TABLE default.flow_records (
  flow_id STRING,
  network_id STRING,
  created_at TIMESTAMP,
  bytes_sent BIGINT
)
USING iceberg
PARTITIONED BY (
  identity(network_id),
  days(created_at)
)
""")

25/04/30 19:01:44 INFO RESTSessionCatalog: Table properties set at catalog level through catalog properties: {}
25/04/30 19:01:44 INFO RESTSessionCatalog: Table properties enforced at catalog level through catalog properties: {}


DataFrame[]

In [4]:
from pyspark.sql.functions import expr
from datetime import datetime, timedelta
import random

# Skewed: net1 has 500,000 records, others have 5,000
data = [
    (f"flow_net1_{i}", "net1", datetime(2025, 4, 30, i % 24), random.randint(1000, 100000))
    for i in range(500_000)
] + [
    (f"flow_net{i}_{j}", f"net{i}", datetime(2025, 4, 30, j % 24), random.randint(1000, 100000))
    for i in range(2, 10) for j in range(500)
]

df = spark.createDataFrame(data, ["flow_id", "network_id", "created_at", "bytes_sent"])


In [5]:
df.writeTo("default.flow_records").append()


25/04/30 19:05:25 INFO CodeGenerator: Code generated in 98.643292 ms
25/04/30 19:05:25 INFO DAGScheduler: Registering RDD 6 (append at NativeMethodAccessorImpl.java:0) as input to shuffle 0
25/04/30 19:05:25 INFO DAGScheduler: Got map stage job 0 (append at NativeMethodAccessorImpl.java:0) with 12 output partitions
25/04/30 19:05:25 INFO DAGScheduler: Final stage: ShuffleMapStage 0 (append at NativeMethodAccessorImpl.java:0)
25/04/30 19:05:25 INFO DAGScheduler: Parents of final stage: List()
25/04/30 19:05:25 INFO DAGScheduler: Missing parents: List()
25/04/30 19:05:25 INFO DAGScheduler: Submitting ShuffleMapStage 0 (MapPartitionsRDD[6] at append at NativeMethodAccessorImpl.java:0), which has no missing parents
25/04/30 19:05:25 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 16.9 KiB, free 434.4 MiB)
25/04/30 19:05:25 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 8.7 KiB, free 434.4 MiB)
25/04/30 19:05:25 INFO Block

In [6]:
spark.sql("""
ALTER TABLE default.flow_records
WRITE DISTRIBUTED BY PARTITION
""")

DataFrame[]

In [7]:
df.writeTo("default.flow_records").overwritePartitions()


25/04/30 19:07:00 INFO DAGScheduler: Registering RDD 9 (overwritePartitions at NativeMethodAccessorImpl.java:0) as input to shuffle 1
25/04/30 19:07:00 INFO DAGScheduler: Got map stage job 2 (overwritePartitions at NativeMethodAccessorImpl.java:0) with 12 output partitions
25/04/30 19:07:00 INFO DAGScheduler: Final stage: ShuffleMapStage 3 (overwritePartitions at NativeMethodAccessorImpl.java:0)
25/04/30 19:07:00 INFO DAGScheduler: Parents of final stage: List()
25/04/30 19:07:00 INFO DAGScheduler: Missing parents: List()
25/04/30 19:07:00 INFO DAGScheduler: Submitting ShuffleMapStage 3 (MapPartitionsRDD[9] at overwritePartitions at NativeMethodAccessorImpl.java:0), which has no missing parents
25/04/30 19:07:00 INFO MemoryStore: Block broadcast_3 stored as values in memory (estimated size 16.9 KiB, free 434.3 MiB)
25/04/30 19:07:00 INFO MemoryStore: Block broadcast_3_piece0 stored as bytes in memory (estimated size 8.7 KiB, free 434.3 MiB)
25/04/30 19:07:00 INFO BlockManagerInfo: Adde

In [12]:
df = df.repartition(8, "network_id", expr("date(created_at)"))



In [15]:
df.writeTo("default.flow_records").overwritePartitions()

25/04/30 19:15:54 INFO DAGScheduler: Registering RDD 29 (overwritePartitions at NativeMethodAccessorImpl.java:0) as input to shuffle 6
25/04/30 19:15:54 INFO DAGScheduler: Got map stage job 9 (overwritePartitions at NativeMethodAccessorImpl.java:0) with 12 output partitions
25/04/30 19:15:54 INFO DAGScheduler: Final stage: ShuffleMapStage 14 (overwritePartitions at NativeMethodAccessorImpl.java:0)
25/04/30 19:15:54 INFO DAGScheduler: Parents of final stage: List()
25/04/30 19:15:54 INFO DAGScheduler: Missing parents: List()
25/04/30 19:15:54 INFO DAGScheduler: Submitting ShuffleMapStage 14 (MapPartitionsRDD[29] at overwritePartitions at NativeMethodAccessorImpl.java:0), which has no missing parents
25/04/30 19:15:54 INFO MemoryStore: Block broadcast_12 stored as values in memory (estimated size 24.2 KiB, free 434.3 MiB)
25/04/30 19:15:54 INFO MemoryStore: Block broadcast_12_piece0 stored as bytes in memory (estimated size 12.0 KiB, free 434.3 MiB)
25/04/30 19:15:54 INFO BlockManagerInf

In [13]:
df.rdd.getNumPartitions()

25/04/30 19:15:40 INFO DAGScheduler: Registering RDD 23 (javaToPython at NativeMethodAccessorImpl.java:0) as input to shuffle 5
25/04/30 19:15:40 INFO DAGScheduler: Got map stage job 8 (javaToPython at NativeMethodAccessorImpl.java:0) with 12 output partitions
25/04/30 19:15:40 INFO DAGScheduler: Final stage: ShuffleMapStage 13 (javaToPython at NativeMethodAccessorImpl.java:0)
25/04/30 19:15:40 INFO DAGScheduler: Parents of final stage: List()
25/04/30 19:15:40 INFO DAGScheduler: Missing parents: List()
25/04/30 19:15:40 INFO DAGScheduler: Submitting ShuffleMapStage 13 (MapPartitionsRDD[23] at javaToPython at NativeMethodAccessorImpl.java:0), which has no missing parents
25/04/30 19:15:40 INFO MemoryStore: Block broadcast_11 stored as values in memory (estimated size 24.2 KiB, free 434.3 MiB)
25/04/30 19:15:40 INFO MemoryStore: Block broadcast_11_piece0 stored as bytes in memory (estimated size 12.0 KiB, free 434.3 MiB)
25/04/30 19:15:40 INFO BlockManagerInfo: Added broadcast_11_piece0

8

In [25]:
df = (
    df
    .withColumn("bucket", (hash("flow_id") % 16).cast("int"))
    .repartition("network_id", expr("date(created_at)"), "bucket")
    .drop("bucket").drop("bucket_hint")  # 🔥 MUST drop before writing to Iceberg
)

df.writeTo("default.flow_records").append()

25/05/01 06:32:48 INFO DAGScheduler: Registering RDD 48 (append at NativeMethodAccessorImpl.java:0) as input to shuffle 12
25/05/01 06:32:48 INFO DAGScheduler: Got map stage job 17 (append at NativeMethodAccessorImpl.java:0) with 12 output partitions
25/05/01 06:32:48 INFO DAGScheduler: Final stage: ShuffleMapStage 35 (append at NativeMethodAccessorImpl.java:0)
25/05/01 06:32:48 INFO DAGScheduler: Parents of final stage: List()
25/05/01 06:32:48 INFO DAGScheduler: Missing parents: List()
25/05/01 06:32:48 INFO DAGScheduler: Submitting ShuffleMapStage 35 (MapPartitionsRDD[48] at append at NativeMethodAccessorImpl.java:0), which has no missing parents
25/05/01 06:32:48 INFO MemoryStore: Block broadcast_21 stored as values in memory (estimated size 24.2 KiB, free 434.3 MiB)
25/05/01 06:32:48 INFO MemoryStore: Block broadcast_21_piece0 stored as bytes in memory (estimated size 12.0 KiB, free 434.3 MiB)
25/05/01 06:32:48 INFO BlockManagerInfo: Added broadcast_21_piece0 in memory on fa5472e5

In [26]:
df.show()

25/05/01 06:32:55 INFO DAGScheduler: Registering RDD 75 (showString at NativeMethodAccessorImpl.java:0) as input to shuffle 21
25/05/01 06:32:55 INFO DAGScheduler: Got map stage job 27 (showString at NativeMethodAccessorImpl.java:0) with 12 output partitions
25/05/01 06:32:55 INFO DAGScheduler: Final stage: ShuffleMapStage 90 (showString at NativeMethodAccessorImpl.java:0)
25/05/01 06:32:55 INFO DAGScheduler: Parents of final stage: List()
25/05/01 06:32:55 INFO DAGScheduler: Missing parents: List()
25/05/01 06:32:55 INFO DAGScheduler: Submitting ShuffleMapStage 90 (MapPartitionsRDD[75] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
25/05/01 06:32:55 INFO MemoryStore: Block broadcast_32 stored as values in memory (estimated size 24.2 KiB, free 434.3 MiB)
25/05/01 06:32:55 INFO MemoryStore: Block broadcast_32_piece0 stored as bytes in memory (estimated size 12.0 KiB, free 434.3 MiB)
25/05/01 06:32:55 INFO BlockManagerInfo: Added broadcast_32_piece0 in me

+-------------+----------+-------------------+----------+
|      flow_id|network_id|         created_at|bytes_sent|
+-------------+----------+-------------------+----------+
| flow_net9_59|      net9|2025-04-30 11:00:00|     42246|
| flow_net9_67|      net9|2025-04-30 19:00:00|     71074|
|flow_net9_183|      net9|2025-04-30 15:00:00|     84031|
|flow_net9_193|      net9|2025-04-30 01:00:00|      8733|
|flow_net9_219|      net9|2025-04-30 03:00:00|      6481|
|flow_net9_221|      net9|2025-04-30 05:00:00|     59891|
|flow_net9_272|      net9|2025-04-30 08:00:00|     42020|
|flow_net9_274|      net9|2025-04-30 10:00:00|     25452|
|flow_net9_350|      net9|2025-04-30 14:00:00|     43967|
|flow_net9_381|      net9|2025-04-30 21:00:00|     41857|
|flow_net9_397|      net9|2025-04-30 13:00:00|     52746|
|flow_net9_419|      net9|2025-04-30 11:00:00|     24289|
|flow_net9_484|      net9|2025-04-30 04:00:00|     37136|
| flow_net1_20|      net1|2025-04-30 20:00:00|     59928|
|flow_net1_113

25/05/01 06:32:56 INFO Executor: Finished task 2.0 in stage 117.0 (TID 227). 4218 bytes result sent to driver
25/05/01 06:32:56 INFO Executor: Finished task 0.0 in stage 117.0 (TID 225). 4218 bytes result sent to driver
25/05/01 06:32:56 INFO TaskSetManager: Finished task 2.0 in stage 117.0 (TID 227) in 36 ms on fa5472e59cd7 (executor driver) (2/6)
25/05/01 06:32:56 INFO TaskSetManager: Finished task 0.0 in stage 117.0 (TID 225) in 38 ms on fa5472e59cd7 (executor driver) (3/6)
25/05/01 06:32:56 INFO Executor: Finished task 4.0 in stage 117.0 (TID 229). 4218 bytes result sent to driver
25/05/01 06:32:56 INFO TaskSetManager: Finished task 4.0 in stage 117.0 (TID 229) in 38 ms on fa5472e59cd7 (executor driver) (4/6)
25/05/01 06:32:56 INFO Executor: Finished task 1.0 in stage 117.0 (TID 226). 4218 bytes result sent to driver
25/05/01 06:32:56 INFO TaskSetManager: Finished task 1.0 in stage 117.0 (TID 226) in 39 ms on fa5472e59cd7 (executor driver) (5/6)
25/05/01 06:32:56 INFO Executor: Fin

In [22]:
df.drop("bucket_hint").drop("bucket")

DataFrame[flow_id: string, network_id: string, created_at: timestamp, bytes_sent: bigint]

In [27]:
spark.sql("""
CREATE TABLE default.flow_records_test_identity (
  flow_id STRING,
  network_id STRING,
  created_at TIMESTAMP,
  bytes_sent BIGINT
)
USING iceberg
PARTITIONED BY (
  network_id,
  days(created_at)
)
""")

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


25/05/01 11:27:14 INFO RESTSessionCatalog: Table properties set at catalog level through catalog properties: {}
25/05/01 11:27:14 INFO RESTSessionCatalog: Table properties enforced at catalog level through catalog properties: {}


DataFrame[]

In [28]:
from pyspark.sql.functions import col, expr, concat_ws, lit, floor, rand
from datetime import datetime

# 50 million rows
N = 50_000_000

df = (
    spark.range(N)
    .withColumn("network_id", expr("CASE WHEN id % 10 = 0 THEN 'net1' ELSE concat('net', (id % 10)) END"))
    .withColumn("flow_id", concat_ws("_", lit("flow"), col("network_id"), col("id")))
    .withColumn("created_at", expr("timestamp_seconds(1714435200 + (id % (60*60*24)))"))  # random seconds on 2025-04-30
    .withColumn("bytes_sent", (rand() * 100000 + 1000).cast("long"))
    .select("flow_id", "network_id", "created_at", "bytes_sent")
)

df.write.mode("overwrite").parquet("/opt/spark/home/iceberg/datasets/flow_50m")

25/05/04 12:05:14 INFO ParquetUtils: Using default output committer for Parquet: org.apache.parquet.hadoop.ParquetOutputCommitter
25/05/04 12:05:14 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
25/05/04 12:05:14 INFO FileOutputCommitter: FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
25/05/04 12:05:14 INFO SQLHadoopMapReduceCommitProtocol: Using user defined output committer class org.apache.parquet.hadoop.ParquetOutputCommitter
25/05/04 12:05:14 INFO FileOutputCommitter: File Output Committer Algorithm version is 1
25/05/04 12:05:14 INFO FileOutputCommitter: FileOutputCommitter skip cleanup _temporary folders under output directory:false, ignore cleanup failures: false
25/05/04 12:05:14 INFO SQLHadoopMapReduceCommitProtocol: Using output committer class org.apache.parquet.hadoop.ParquetOutputCommitter
25/05/04 12:05:14 INFO CodeGenerator: Code generated in 15.539709 ms
25/05/04 12:05:14 INFO Spa

In [29]:
spark